In [ ]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import copy
import GPUtil
from collections import defaultdict
from analysis import utils as au
from analysis import plotting
from data import utils as du
from data import se3_diffuser
from data import r3_diffuser
from data import so3_diffuser
from model import loss
from model import reverse_se3_diffusion
import tree
from data import rosetta_data_loader
from data import digs_data_loader
from experiments import train_se3_diffusion
from openfold.utils import rigid_utils as ru
from scipy.spatial.transform import Rotation

from omegaconf import DictConfig, OmegaConf
import importlib

# Enable logging
import logging
import sys
date_strftime_format = "%Y-%m-%y %H:%M:%S"
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(asctime)s %(message)s", datefmt=date_strftime_format)

In [ ]:
importlib.reload(rosetta_data_loader)
importlib.reload(digs_data_loader)
importlib.reload(se3_diffuser)
importlib.reload(so3_diffuser)
importlib.reload(r3_diffuser)
importlib.reload(du)
importlib.reload(loss)
importlib.reload(reverse_se3_diffusion)
importlib.reload(train_se3_diffusion)

In [ ]:
inference_conf_path = '../config/inference.yaml'
inference_conf = OmegaConf.load(inference_conf_path)

In [ ]:
inference_conf.ckpt_dir = '../pkl_jar/ckpt/subset_1000_31D_08M_2022Y_14h_59m_25s'
inference_conf.default_conf_path = '../config/base.yaml'
inference_conf.output_dir = '../results'

print(OmegaConf.to_yaml(inference_conf))

In [ ]:
class Sampler:

    def __init__(self, conf: DictConfig):
        self._log = logging.getLogger(__name__)
        self._infer_conf = conf
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

        self._load_ckpt()

    def _load_ckpt(self):
        ckpt_dir = self._infer_conf.ckpt_dir
        if len(os.listdir(ckpt_dir)) != 1:
            raise ValueError(f'Ambiguous ckpt in {ckpt_dir}')
        ckpt_name = os.listdir(ckpt_dir)[0]
        self._output_dir = os.path.join(self._infer_conf.output_dir, os.path.basename(ckpt_dir))
        os.makedirs(self._output_dir, exist_ok=True)
        ckpt_path = os.path.join(ckpt_dir, ckpt_name)
        self._log.info(f'Loading checkpoint from {ckpt_path}')
        self._log.info(f'Saving results to {self._output_dir}')

        # Read checkpoint and create experiment.
        ckpt_pkl = du.read_pkl(ckpt_path)
        ckpt_conf = ckpt_pkl['conf']
        ckpt_model = ckpt_pkl['model']
        default_conf = OmegaConf.load(self._infer_conf.default_conf_path)
        self._exp_conf = OmegaConf.merge(default_conf, ckpt_conf)
        self._exp_conf.experiment.data_location = 'digs'
        self._exp_conf.data.digs.cache_dir = 'pkl_jar/'
        
        self.exp = train_se3_diffusion.Experiment(conf=self._exp_conf)
        self.exp._model = self.exp._model.to(self.device)  
        self.exp.model.load_state_dict(ckpt_model)
        
        self.diffuser = self.exp.diffuser
        
        self.scale_factor = self.exp._data_conf.scale_factor
    
    def sample(self, batch_size, num_res, file_prefix=None):
        
        # Initialize data
        rigids_init = self.diffuser.sample_ref(
            num_res * batch_size 
        ).to_tensor_7().reshape((batch_size, num_res, 7))
        init_feats = {
            'rigids_t': rigids_init,
            'res_mask': torch.ones((batch_size, num_res)),
            'res_idx': torch.tile(
                torch.arange(1, num_res+1)[None], (batch_size, 1)),
        }
        init_feats = tree.map_structure(
            lambda x: x.to(self.device), init_feats)
        
        # Run inference
        infer_out = sampler.exp.inference_fn(
            init_feats, add_noise=True)
        sampled_rigids = infer_out[0]
        final_samples = sampled_rigids[-1]
        traj_samples = torch.stack(sampled_rigids).transpose(0, 1)
        
        # Save each sample
        pdb_name = f'len_{num_res}_sample.pdb'
        traj_name = f'len_{num_res}_trajectory.pdb'
        if file_prefix is not None:
            pdb_name = f'{file_prefix}_' + pdb_name
            traj_name = f'{file_prefix}_' + traj_name
        for i in range(batch_size):
            self.save_ca_pdb(final_samples[i][..., 4:], pdb_name)
            self.save_ca_pdb(traj_samples[i][..., 4:], traj_name)
        return sampled_rigids
        
    def save_ca_pdb(self, ca_pos, pdb_name, rescale=True):
        if rescale:
            ca_pos = ca_pos * self.scale_factor
        save_path = os.path.join(self._output_dir, pdb_name)
        return au.write_prot_to_pdb(
            du.move_to_np(ca_pos),
            save_path
        )
    
    
        
    

In [ ]:
sampler = Sampler(inference_conf)

In [ ]:
num_res = 120
batch_size = 16

In [ ]:
sampled_outputs = sampler.sample(batch_size, num_res, file_prefix='test')

In [ ]:
final_samples = sampled_outputs[-1]

In [ ]:
sample_i = final_samples[0]

In [ ]:
sample_i

In [62]:
rigids_init = sampler.diffuser.sample_ref(
    num_res * batch_size 
).to_tensor_7().reshape((batch_size, num_res, 7))
init_feats = {
    'rigids_t': rigids_init,
    'res_mask': torch.ones((batch_size, num_res)),
    'res_idx': torch.tile(
        torch.arange(1, num_res+1)[None], (batch_size, 1)),
}
init_feats = tree.map_structure(lambda x: x.to(sampler.device), init_feats)
infer_out = sampler.exp.inference_fn(
    init_feats, add_noise=True)
sampled_rigids = infer_out[0]

In [64]:
sampled_rigids[-1]

torch.Size([16, 120, 7])

In [ ]:
# Save final design as PDB.
ca_pos = final_rigid[:, 4:] * scale_factor
num_res = du.move_to_np(torch.sum(res_mask))
file_path = f'../samples/pdb/{os.path.basename(ckpt_dir)}/len_{num_res}_sample.pdb'
file_dir = os.path.dirname(file_path)
os.makedirs(file_dir, exist_ok=True)

save_path = au.write_prot_to_pdb(
    du.move_to_np(ca_pos),
    file_path
)
print(f'Written to {save_path}')

In [3]:
base_conf_path = '../config/base.yaml'
base_conf = OmegaConf.load(base_conf_path)

# Load model checkpoint
# ckpt_dir = '../pkl_jar/ckpt/subset_1000_exp_lin_31D_08M_2022Y_15h_49m_56s'
# ckpt_dir = '../pkl_jar/ckpt/subset_1000_lin_lin_31D_08M_2022Y_15h_00m_54s'
# ckpt_dir = '../pkl_jar/ckpt/subset_100_01D_09M_2022Y_02h_47m_59s'
# ckpt_dir = '../pkl_jar/ckpt/baseline_01D_09M_2022Y_05h_20m_34s'
ckpt_dir = '../pkl_jar/ckpt/subset_1000_31D_08M_2022Y_14h_59m_25s'
# ckpt_dir = '../pkl_jar/ckpt/subset_1000_lin_log_4_01D_09M_2022Y_02h_47m_59s'
# ckpt_dir = '../pkl_jar/ckpt/baseline_01D_09M_2022Y_10h_45m_05s'
# ckpt_dir = '../pkl_jar/ckpt/pdb_lin_log_3_01D_09M_2022Y_11h_25m_32s'


if len(os.listdir(ckpt_dir)) != 1:
    raise ValueError(f'Ambiguous ckpt in {ckpt_dir}')
ckpt_path = os.path.join(
    ckpt_dir, os.listdir(ckpt_dir)[0])
print(ckpt_path)
ckpt_pkl = du.read_pkl(ckpt_path)
ckpt_conf = ckpt_pkl['conf']
ckpt_model = ckpt_pkl['model']


conf = OmegaConf.merge(base_conf, ckpt_conf)
conf.experiment.data_location = 'digs'
conf.data.digs.cache_dir = '../pkl_jar/'

# print(OmegaConf.to_yaml(ckpt_conf))

../pkl_jar/ckpt/subset_1000_31D_08M_2022Y_14h_59m_25s/epoch_51.pkl


In [4]:
exp = train_se3_diffusion.Experiment(conf=conf)
train_loader, train_sampler, valid_loader, valid_sampler = exp.create_digs_dataset(0, 1)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
exp._model = exp._model.to(device)
exp.model.load_state_dict(ckpt_model)

INFO: Number of model parameters 3488030
INFO: Using cached IGSO3.
INFO: Loaded data at ../pkl_jar/dataset_5.0_260_60_80.0_100_2020-Apr-30_1000.pkl
INFO: Loaded data at ../pkl_jar/dataset_5.0_260_60_80.0_100_2020-Apr-30_1000.pkl


<All keys matched successfully>

In [5]:
# Sample data point to initializez.
data_iter = iter(train_loader)
raw_data_feats = next(data_iter)

In [6]:
# Preprocess init data
exp_diffuser = exp.diffuser
# batch_idx = 2
# data_feats = tree.map_structure(lambda x: x[batch_idx], raw_data_feats)
data_feats = traw_data_feats
res_mask = data_feats['res_mask']
res_idx = data_feats['res_idx']
rigids_0 = data_feats['rigids_0']

# rigids_0 = rigids_0[torch.where(res_mask.bool())]
# res_mask = res_mask[torch.where(res_mask.bool())]

num_res = res_mask.shape[0]
rigids_init = exp_diffuser.sample_ref(
    num_res,
    impute=ru.Rigid.from_tensor_7(rigids_0)
).to_tensor_7()
rigids_init *= res_mask[:, None]
init_feats = {
    'rigids_t': rigids_init,
    'res_mask': res_mask,
    'res_idx': res_idx,
#     'res_idx': res_idx[torch.where(res_mask.bool())],
    't': torch.tensor(1.0),
    'rot_score_norm': data_feats['rot_score_norm'],
    'trans_score_norm': data_feats['trans_score_norm'],
}

num_res = res_mask.shape[0]
init_feats = tree.map_structure(lambda x: x[None].to(device), init_feats)
print(f'Sample length {torch.sum(res_mask)}')

Sample length 206


In [ ]:
# Run inference
infer_out = exp.inference_fn(init_feats, add_noise=True)
sample_rigids = infer_out[0]

In [ ]:
# Plot out SE(3)
final_rigid = sample_rigids[-1][0]
f, axes = plt.subplots(1, 3, figsize=[24, 8], subplot_kw=dict(projection="3d"))
scale_factor = conf.data.digs.scale_factor
ax_lim = 15.
plotting.viz_frames(init_feats['rigids_t'][0], res_mask, axes[0], title='Initialization', scale_factor=scale_factor, ax_lim=ax_lim)
plotting.viz_frames(final_rigid, res_mask, axes[1], title='Sample', scale_factor=scale_factor, ax_lim=ax_lim)
plotting.viz_frames(rigids_0, res_mask, axes[2], title='Ground truth', scale_factor=scale_factor, ax_lim=ax_lim)

In [ ]:
# Calculate FAPE and Ca-RMSD metrics.
fape_loss = loss.rigids_fape(
    final_rigid,
    rigids_0.to(device),
    res_mask.to(device),
    length_scale=scale_factor)
rmsd_loss, aligned_pred_ca, gt_ca, align_rot_mat, align_trans, reflection = loss.rigids_ca_rmsd(
    final_rigid,
    rigids_0.to(device),
    res_mask.to(device),
    length_scale=scale_factor,
    return_align=True)
align_rot_euler = Rotation.from_matrix(align_rot_mat).as_euler('xyz', degrees=True)
print(f'FAPE: {fape_loss:2.4f}\nCa-RMSD: {rmsd_loss:2.4f}')
print(f'Alignment rotation: {align_rot_euler}\nAlignment translation: {align_trans[:,0]}\nAlignment reflection: {reflection}')


In [ ]:
# Visualize aligned Ca structures
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection='3d')

plotting.plt_3d(aligned_pred_ca, ax, color='r', s=100, mode='scatter')
plotting.plt_3d(aligned_pred_ca, ax, color='r', mode='line')
plotting.plt_3d(gt_ca, ax, color='b', s=100, mode='scatter')
plotting.plt_3d(gt_ca, ax, color='b', mode='line')

In [ ]:
# Save final design as PDB.
ca_pos = final_rigid[:, 4:] * scale_factor
num_res = du.move_to_np(torch.sum(res_mask))
file_path = f'../samples/pdb/{os.path.basename(ckpt_dir)}/len_{num_res}_sample.pdb'
file_dir = os.path.dirname(file_path)
os.makedirs(file_dir, exist_ok=True)

save_path = au.write_prot_to_pdb(
    du.move_to_np(ca_pos),
    file_path
)
print(f'Written to {save_path}')

In [ ]:
# Save trajectory as a gif
save_path = 'dev_diffusion_linear.gif'
sample_traj = torch.stack(sample_rigids)[:, 0]
ax_lim = 15
plotting.write_traj(
    sample_traj, res_mask, save_path, ax_lim=ax_lim, scale_factor=scale_factor
)